In [20]:
# Import the required libraries and dependencies
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import random
from MCForecastTools import MCSimulation
import ipywidgets as widgets

%matplotlib inline

In [21]:
ticker_list_file = open('Resources/stock_universe.json')
ticker_list = json.load(ticker_list_file)

In [22]:
# Load the environment variables from the .env file
#by calling the load_dotenv function
load_dotenv()

True

In [23]:
# Set the variables for the Alpaca API and secret keys
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca tradeapi.REST object
alpaca = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version="v2")

In [24]:
#Create a checkbox for each of the stocks the user may select
stock_widgets = []
for stock in ticker_list:
    stock_widgets.append(widgets.Checkbox(description=stock, width=10))

In [25]:
#Ask how many years the user is willing to hold the stocks at maximum
max_years_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=50,
    step=1,
    width='100%',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='',
    slider_color='white'
)

max_years_label = widgets.Label('Maximum Years Willing to Hold Stocks:', style={'description_width': 'initial'})

max_years_widget = widgets.HBox([max_years_label, max_years_slider])

In [26]:
#Design a submit button
submit_stocks_button = widgets.Button(
    description='Submit Stocks',
    disabled=False,
    button_style='success',
    tooltip='Submit Selection',
)

submit_money_button = widgets.Button(
    description='Submit Max Years and Investment Amounts',
    disabled=False,
    button_style='success',
    tooltip='Submit Max Years and Investment Amounts',
)

clear_button = widgets.Button(
    description='Clear Stock Selections',
    button_style='danger',
    tooltop='Clear Stock Selections')

#When the user clicks the submit button, we print the names of the stocks selected and add the selected stock to our list of selected stocks

def on_submit_stocks(button):
    num_selected=0
    global stocks_selected
    stocks_selected=[]
    for widget in stock_widgets:
        if widget.value:
            num_selected=num_selected + 1
            stocks_selected.append(str(widget.description))
    if num_selected > 3:
        print("You have selected too many stocks. Please select three stocks and re-submit.")
    elif num_selected < 3:
        print("You have not selected enough stocks. Please select three stocks and re-submit.")
    else:
        print("You have selected the following stocks:")
        for i in range(len(stocks_selected)):
            print(stocks_selected[i])
            
def on_submit_money(button):
    print("You are willing to hold these stocks for " + str(max_years_slider.value) + " years.")
    global max_years, stock_1_investment, stock_2_investment, stock_3_investment
    max_years = max_years_slider.value
    stock_1_investment = stock_1_money.value
    stock_2_investment = stock_2_money.value
    stock_3_investment = stock_3_money.value
    
def clear(button):
    for widget in stock_widgets:
        widget.value=False

submit_stocks_button.on_click(on_submit_stocks)
submit_money_button.on_click(on_submit_money)
clear_button.on_click(clear)

In [27]:
#Design the UI for the user input

stock_layout = widgets.Layout(border='3px solid black',
                    width='100%',
                    height='500px',
                    flex_flow='row wrap',
                    flex_shrink='True',
                    display='inline-flex')

ui = widgets.VBox([widgets.HBox(children=[stock_widgets[a] for a in range((len(stock_widgets)))], layout=stock_layout), 
                   widgets.HBox([submit_stocks_button, clear_button])])

display(ui)

In [28]:
stock_1_money = widgets.IntText(disabled=False)
stock_2_money = widgets.IntText(disabled=False)
stock_3_money = widgets.IntText(disabled=False)
stock_1_money_label = widgets.Label('How much money do you plan to invest in ' + stocks_selected[0])
stock_2_money_label = widgets.Label('How much money do you plan to invest in ' + stocks_selected[1])
stock_3_money_label = widgets.Label('How much money do you plan to invest in ' + stocks_selected[2])
stock_1_money_widget = widgets.HBox([stock_1_money_label, stock_1_money])
stock_2_money_widget = widgets.HBox([stock_2_money_label, stock_2_money])
stock_3_money_widget = widgets.HBox([stock_3_money_label, stock_3_money])

ui_2 = widgets.VBox([max_years_widget, stock_1_money_widget, stock_2_money_widget, stock_3_money_widget, submit_money_button])
display(ui_2)

In [29]:
#Set the tickers to three random stocks (for now) and random max number of years
amount_invested = stock_1_investment+stock_2_investment+stock_3_investment
inv_weights = [stock_1_investment/amount_invested, stock_2_investment/amount_invested, stock_3_investment/amount_invested]
    
# Set timeframe to 1Day
timeframe = "1Day"

# Format current date as ISO format
# Set both the start and end date at the date of your prior weekday 
# This will give you the closing price of the previous trading day
# Alternatively you can use a start and end date of 2020-08-07
start_date = pd.Timestamp("2017-08-07", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-08-07", tz="America/New_York").isoformat()

In [30]:
# Use the Alpaca get_bars function to get current closing prices the portfolio
# Be sure to set the `df` property after the function to format the response object as a DataFrame
stocks_df = alpaca.get_bars(
    stocks_selected,
    timeframe,
    start=start_date,
    end=end_date
).df
stocks_df.head()

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2017-08-07 04:00:00+00:00,59.00,59.1214,58.67,58.91,598256,5282,58.860697,AJG
2017-08-08 04:00:00+00:00,58.78,58.9700,58.68,58.81,533651,5117,58.804521,AJG
2017-08-09 04:00:00+00:00,58.72,59.0400,58.72,58.90,430894,4151,58.876255,AJG
2017-08-10 04:00:00+00:00,58.65,58.9700,58.43,58.61,924251,5169,58.614808,AJG
2017-08-11 04:00:00+00:00,58.63,58.9800,58.35,58.42,546243,4489,58.516021,AJG


In [31]:
# Reorganize the DataFrame
# Separate ticker data
stock_1 = stocks_df[stocks_df['symbol']==stocks_selected[0]].drop('symbol', axis=1)
stock_2 = stocks_df[stocks_df['symbol']==stocks_selected[1]].drop('symbol', axis=1)
stock_3 = stocks_df[stocks_df['symbol']==stocks_selected[2]].drop('symbol', axis=1)

# Concatenate the ticker DataFrames
stocks_df = pd.concat([stock_1, stock_2, stock_3], axis=1, keys=stocks_selected)

# Review the first 5 rows of the Alpaca DataFrame
stocks_df.head()

AJG                                             \
                            open     high    low  close  volume trade_count   
timestamp                                                                     
2017-08-07 04:00:00+00:00  59.00  59.1214  58.67  58.91  598256        5282   
2017-08-08 04:00:00+00:00  58.78  58.9700  58.68  58.81  533651        5117   
2017-08-09 04:00:00+00:00  58.72  59.0400  58.72  58.90  430894        4151   
2017-08-10 04:00:00+00:00  58.65  58.9700  58.43  58.61  924251        5169   
2017-08-11 04:00:00+00:00  58.63  58.9800  58.35  58.42  546243        4489   

                                        DUK                    ...           \
                                vwap   open     high      low  ...   volume   
timestamp                                                      ...            
2017-08-07 04:00:00+00:00  58.860697  86.46  86.5500  86.0000  ...  2026377   
2017-08-08 04:00:00+00:00  58.804521  86.10  86.6100  86.0100  ...  2103248   
2017-08-09 04:00:00+00:00  58.876255  86.76  86.9000  86.0100  ...  2115384   
2017-08-10 04:00:00+00:00  58.614808  86.20  86.5999  85.7617  ...  2140907   
2017-08-11 04:00:00+00:00  58.516021  86.47  86.4700  85.1600  ...  2516942   

                                                      F                       \
                          trade_count       vwap   open   high    low  close   
timestamp                                                                      
2017-08-07 04:00:00+00:00       16654  86.253406  10.94  10.98  10.90  10.92   
2017-08-08 04:00:00+00:00       15746  86.377587  10.95  11.00  10.88  10.89   
2017-08-09 04:00:00+00:00       16881  86.304284  10.88  10.95  10.83  10.92   
2017-08-10 04:00:00+00:00       19103  86.230069  10.88  10.91  10.77  10.77   
2017-08-11 04:00:00+00:00       20881  85.666246  10.79  10.84  10.76  10.77   

                                                            
                             volume trade_count       vwap  
timestamp                                                   
2017-08-07 04:00:00+00:00  35015876       50274  10.924778  
2017-08-08 04:00:00+00:00  26870382       43601  10.934841  
2017-08-09 04:00:00+00:00  32157172       57040  10.886567  
2017-08-10 04:00:00+00:00  31923827       52389  10.824857  
2017-08-11 04:00:00+00:00  32320117       56634  10.788049  

[5 rows x 21 columns]

In [32]:
#Initialize two variables to track the best year and best median based on MC Simulations
best_mean = 0
list_means = []
best_year = 0

In [33]:
for year in range(max_years):
    MC = MCSimulation(
        portfolio_data = stocks_df,
        weights = inv_weights,
        num_simulation=500,
        num_trading_days = 252*year
        )
    MC.calc_cumulative_return()
    statistics = MC.summarize_cumulative_return()
    mean = statistics[1]
    list_means.append(mean)
    if mean > best_mean:
        best_mean = mean
        best_year = year+1
    #add in volatility tracking of some kind

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.


/Users/rachelphillips/Desktop/FinTech-Workspace/FinTech-Project-1/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Monte Carlo simulation number 340.
Running Mon

In [34]:
print(f"Based on our analysis, we recommend holding your stocks for"
      f" {best_year} years.")

Based on our analysis, we recommend holding your stocks for 21 years.
